# Joins

When we explored our different datasets. We need to merge the different data sources. To be able to do it we need keys to join the data. To understand how to join data in Vertica ML Python, let's look at an example. 

Let's use the US Flights 2015 datasets. 3 datasets are available.
We have information on the different flights.

In [120]:
from vertica_ml_python import *
flights  = vDataFrame("flights")
print(flights)

,departure_delay,origin_airport,scheduled_departure,airline,destination_airport,arrival_delay
0,-9,11433,2015-10-01 10:09:00,EV,10135,-2
1,-3,10397,2015-10-01 10:27:00,EV,10135,-14
2,-4,13930,2015-10-01 13:57:00,EV,10135,6
3,-3,11433,2015-10-01 14:02:00,EV,10135,-8
4,0,10397,2015-10-01 14:44:00,EV,10135,-1
,...,...,...,...,...,...


<object>  Name: flights, Number of rows: 4068736, Number of columns: 6


A dataset on the airports information is also available.

In [106]:
airports = vDataFrame("airports")
print(airports)

,AIRPORT,LATITUDE,CITY,STATE,LONGITUDE,COUNTRY,IATA_CODE
0,Lehigh Valley International Airport,40.65236,Allentown,PA,-75.4404,USA,ABE
1,Abilene Regional Airport,32.41132,Abilene,TX,-99.6819,USA,ABI
2,Albuquerque International Sunport,35.04022,Albuquerque,NM,-106.60919,USA,ABQ
3,Aberdeen Regional Airport,45.44906,Aberdeen,SD,-98.42183,USA,ABR
4,Southwest Georgia Regional Airport,31.53552,Albany,GA,-84.19447,USA,ABY
,...,...,...,...,...,...,...


<object>  Name: airports, Number of rows: 322, Number of columns: 7


And we also have access to the airlines names.

In [107]:
airlines = vDataFrame("airlines")
print(airlines)

,AIRLINE,IATA_CODE
0,American Airlines Inc.,AA
1,Alaska Airlines Inc.,AS
2,JetBlue Airways,B6
3,Delta Air Lines Inc.,DL
4,Atlantic Southeast Airlines,EV
,...,...


<object>  Name: airlines, Number of rows: 14, Number of columns: 2


We can notice that each dataset has a primary or secondary key to join the data. For example we can join the 'flights' dataset to the 'airlines' and 'airport' datasets using the corresponding IATA code. 

To join datasets in Vertica ML Python, use the 'join' method of the vDataFrame.

In [108]:
help(vDataFrame.join)

Help on function join in module vertica_ml_python.vdataframe:

join(self, input_relation, on:dict={}, how:str='natural', expr1:list=['*'], expr2:list=['*'])
    ---------------------------------------------------------------------------
    Joins the vDataFrame with another one or an input relation.
    
    Parameters
    ----------
    input_relation: str/vDataFrame
            Relation used to do the merging.
    on: dict, optional
            Dictionary of all the different keys. The dict must be similar to the following:
            {"relationA_key1": "relationB_key1" ..., "relationA_keyk": "relationB_keyk"}
            where relationA is the current vDataFrame and relationB is the input relation
            or the input vDataFrame.
    how: str, optional
            Join Type.
                    left    : Left Join.
                    right   : Right Join.
                    cross   : Cross Join.
                    full    : Full Outer Join.
                    natural : Natu

Let's use a left join to merge the 'airlines' dataset and the 'flights' dataset.

In [121]:
flights = flights.join(airlines,
                       how = "left",
                       on = {"airline": "IATA_CODE"},
                       expr2 = ["AIRLINE AS airline_long"])
print(flights)

,destination_airport,scheduled_departure,departure_delay,arrival_delay,origin_airport,airline,airline_long
0,10135,2015-10-01 10:09:00,-9,-2,11433,EV,Atlantic Southeast Airlines
1,10135,2015-10-01 10:27:00,-3,-14,10397,EV,Atlantic Southeast Airlines
2,10135,2015-10-01 13:57:00,-4,6,13930,EV,Atlantic Southeast Airlines
3,10135,2015-10-01 14:02:00,-3,-8,11433,EV,Atlantic Southeast Airlines
4,10135,2015-10-01 14:44:00,0,-1,10397,EV,Atlantic Southeast Airlines
,...,...,...,...,...,...,...


<object>  Name: join, Number of rows: 4068736, Number of columns: 7


Let's use two left joins to get the information on the origin and destination airports.

In [126]:
flights = flights.join(airports,
                       how = "left",
                       on = {"origin_airport": "IATA_CODE"},
                       expr2 = ["LATITUDE AS origin_lat",
                                "LONGITUDE AS origin_lon"])
flights = flights.join(airports,
                       how = "left",
                       on = {"destination_airport": "IATA_CODE"},
                       expr2 = ["LATITUDE AS destination_lat",
                                "LONGITUDE AS destination_lon"])
print(flights)

,destination_airport,scheduled_departure,departure_delay,arrival_delay,origin_airport,airline,airline_long,origin_lat,origin_lon,destination_lat,destination_lon
0,BRW,2015-01-02 17:28:00,-6,-18,ANC,AS,Alaska Airlines Inc.,61.17432,-149.99619,71.28545,-156.766
1,BRW,2015-01-03 17:21:00,-7,-11,SCC,AS,Alaska Airlines Inc.,70.19476,-148.46516,71.28545,-156.766
2,BRW,2015-01-01 17:21:00,-7,-9,SCC,AS,Alaska Airlines Inc.,70.19476,-148.46516,71.28545,-156.766
3,BRW,2015-01-02 17:21:00,-7,2,SCC,AS,Alaska Airlines Inc.,70.19476,-148.46516,71.28545,-156.766
4,BRW,2015-01-03 17:28:00,-7,-20,ANC,AS,Alaska Airlines Inc.,61.17432,-149.99619,71.28545,-156.766
,...,...,...,...,...,...,...,...,...,...,...


<object>  Name: join, Number of rows: 4068736, Number of columns: 11


Splitting the data into different tables is very important to avoid duplicated information and to save data storage. Just imagine writing the longitude and the latitude of the destination and origin airports for each flight. It will add too much duplicates and it can drastically increase the data volume.

Cross Joins are special. They don't need a key and they are used to do mathematical operations. Let's now use a cross join of the 'airports' dataset on itself to compute the distance between all the different airports.

In [112]:
distances = airports.join(airports, 
                          how = "cross", 
                          expr1 = ["IATA_CODE AS airport1", 
                                   "LATITUDE AS airport1_latitude", 
                                   "LONGITUDE AS airport1_longitude"],
                          expr2 = ["IATA_CODE AS airport2", 
                                   "LATITUDE AS airport2_latitude", 
                                   "LONGITUDE AS airport2_longitude"])
distances.filter("airport1 != airport2")
distances.eval("distance", 
    "DISTANCE(airport1_latitude, airport1_longitude, airport2_latitude, airport2_longitude)")

322 element(s) was/were filtered


,airport1,airport1_latitude,airport1_longitude,airport2,airport2_latitude,airport2_longitude,distance
0,ABE,40.65236,-75.4404,ABI,32.41132,-99.6819,2341.90022515853
1,ABE,40.65236,-75.4404,ABQ,35.04022,-106.60919,2791.44167745523
2,ABE,40.65236,-75.4404,ABR,45.44906,-98.42183,1934.49820074978
3,ABE,40.65236,-75.4404,ABY,31.53552,-84.19447,1281.62374218022
4,ABE,40.65236,-75.4404,ACK,41.25305,-70.06018,456.66493443057
,...,...,...,...,...,...,...


<object>  Name: join, Number of rows: 103362, Number of columns: 7

We now know how to merge different datasets together. In the next lesson, we will look at other interesting concepts like handling duplicates.